In [2]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

In [8]:
gds.run_cypher("""
MATCH (r:Respondent)-[rel:CHOSE_ALT]->(qa:QuestionAlternative)
SET rel.position = toInteger(qa.position)
""")

""


In [31]:
graph_name = 'studentQuestionAlternativeHypertuning'

if gds.graph.exists(graph_name).any():
    gds.graph.drop(gds.graph.get(graph_name))

G, result = gds.graph.project(
    graph_name,
    ["Student", "QuestionAlternative"],
    {"CHOSE_ALT": { "properties": "position" }}
)

print(f"The projection took {result['projectMillis']} ms")
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")

The projection took 79 ms
Graph 'studentQuestionAlternativeHypertuning' node count: 4574
Graph 'studentQuestionAlternativeHypertuning' node labels: ['Student', 'QuestionAlternative']


In [18]:
result = gds.fastRP.mutate(
    G,
    mutateProperty='embedding',
    randomSeed=42,
    embeddingDimension=256,
    iterationWeights=[0.8, 1],
    relationshipWeightProperty='position'
)
print(f"Number of embedding vectors produced: {result['nodePropertiesWritten']}")

FastRP:   0%|          | 0/100 [00:00<?, ?%/s]

Number of embedding vectors produced: 4574


In [26]:
result = gds.knn.stream(
    G,
    topK=1,
    nodeProperties=["embedding"],
    randomSeed=42,
    concurrency=1,
    sampleRate=1.0,
    deltaThreshold=0.001,
    # writeRelationshipType="SIMILAR_STUDENTS",
    # writeProperty="score",
)

print(set(result['similarity']))

{0.0, 1.0, 0.4814271330833435, 0.4827234447002411, 0.4780368506908417, 0.9678933620452881, 0.9625371694564819, 0.9598968029022217, 0.9985170364379883}


In [34]:
result = gds.louvain.stream(
    G,
    relationshipWeightProperty="position",
)

print(set(result['communityId']))
print(len(set(result['communityId'])))

Louvain:   0%|          | 0/100 [00:00<?, ?%/s]

{2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 236

In [37]:
result = gds.labelPropagation.stream(
    G,
)

print(set(result['communityId']))
print(len(set(result['communityId'])))

{13068, 13070, 13071, 13072, 13073, 13074, 13076, 13077, 13078, 13079, 13080, 13081, 13082, 13083, 13085, 13086, 13088, 13089, 13090, 13092, 13093, 13094, 13095, 13097, 13098, 13099, 13100, 13102, 13103, 13104, 13106, 13107, 13108, 13109, 13110, 13111, 13112, 13114, 13115, 13116, 13117, 13119, 13120, 13121, 13122, 13123, 13125, 13126, 13127, 13128, 13129, 13131, 13132, 13133, 13134, 13135, 13138, 13139, 13140, 13141, 13142, 13144, 13145, 13146, 13147, 13148, 13149, 13150, 13151, 13152, 13153, 13154, 13155, 13156, 13157, 13158, 13159, 13160, 13161, 13164, 13165, 13166, 13168, 13169, 13170, 13171, 13172, 13178, 13179, 13181, 13182, 13183, 13184, 13185, 13187, 13188, 13189, 13190, 13191, 13193, 13194, 13195, 13196, 13197, 13199, 13200, 13201, 13202, 13203, 13205, 13206, 13207, 13208, 13209, 13211, 13212, 13213, 13214, 13215, 13217, 13218, 13219, 13220, 13221, 13222, 13223, 13224, 13225, 13226, 13227, 13228, 13230, 13231, 13232, 13233, 13234, 13236, 13237, 13238, 13239, 13240, 13242, 13243

In [41]:
result = gds.wcc.stream(
    G,
    relationshipWeightProperty="position",
)
print(set(result['componentId']))
print(len(set(result['componentId'])))

{0, 2202, 2355, 2356, 2357, 2358, 2359, 2398, 2399, 2400, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2410, 2411, 2412, 2413, 2415, 2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565, 2566, 2567, 2568, 2569, 2570, 2571, 2572, 2573, 2574, 2575, 2576, 2577, 2578, 2579, 2580, 2581, 2582, 2583, 2584, 2585, 2586, 2587, 2588, 2589, 2590, 2591, 2592, 2593, 2594, 2595, 2596, 2597, 2600, 2609, 2610, 2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2620, 2621, 2622, 2623, 2624, 2625, 2626, 2627, 2628, 2629, 2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639, 2640, 2641, 2642, 2643, 2644, 2645, 2646, 2647, 2648, 2649, 2650, 2651, 2652, 2653, 2654, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2669, 2670, 2671, 2672, 2673, 2674, 2675, 2676, 2677, 

In [42]:
result = gds.beta.modularityOptimization.stream(
    G,
)

print(set(result['communityId']))
print(len(set(result['communityId'])))

{2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 236

In [46]:
result = gds.alpha.maxkcut.stream(
    G,
    k=10,
    relationshipWeightProperty="position",
)

print(set(result['communityId']))
print(len(set(result['communityId'])))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
10


```
CALL gds.alpha.maxkcut.stream(
    'studentQuestionAlternativeHypertuning',
    {k: 12, relationshipWeightProperty: 'position'}
) YIELD nodeId, communityId
WITH nodeId, communityId
MATCH (n:Student) WHERE id(n) = nodeId
SET n.maxKcutId = communityId
RETURN n
```

```
MATCH (s:Student) RETURN s.maxKcutId, count(s) as numberOfStudents
ORDER BY numberOfStudents DESC
```